In [63]:
import pandas as pd
import sys
from pathlib import Path
import os
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sentence_transformers import SentenceTransformer
import string
import re
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", 100)

nlp = spacy.load("en_core_web_lg")

# Add project root to path to enable imports
# If running from notebooks/ directory, go up one level to project root
sys.path.append('..')

from data.shared import CCData

In [64]:
data = pd.read_csv("../data/_PROD_annotations__202511050640.csv")


data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15538 entries, 0 to 15537
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               15538 non-null  int64 
 1   ipeds_code       15538 non-null  int64 
 2   level            15538 non-null  object
 3   start_year       15538 non-null  int64 
 4   end_year         15538 non-null  int64 
 5   university_name  15538 non-null  object
 6   content          15538 non-null  object
 7   type             15538 non-null  object
dtypes: int64(4), object(4)
memory usage: 971.3+ KB


In [65]:
# print entire column - not truncated
pd.set_option('display.max_columns', None)

print(data.iloc[0]['content'])

df = data.copy()

[
  {
    "Department": "VALUES, TECHNOLOGY",
    "Number": "151",
    "Title": "SCIENCE AND SOCIETY",
    "Description": "trol, the H-bomb decision and tactical weapons, disputes over nuclear testing and fallout, the development of deterrence and its critics, the roles of scientists and the strategy community, and various peace movements.",
    "Prerequisites": "",
    "Credits": "4-5",
    "Teacher": {
      "Honorific": "",
      "Name": "Bernstein",
      "Degree": "",
      "Position": "Professor"
    }
  },
  {
    "Department": "VALUES, TECHNOLOGY",
    "Number": "165",
    "Title": "Technology and Musical Expression -",
    "Description": "Interrelations between music-making technologies and the music written for them in Western classical music from the 18th century to the present. (DR:2 or DR:8) for is bra ito",
    "Prerequisites": "",
    "Credits": "4",
    "Teacher": {
      "Honorific": "",
      "Name": "Staff",
      "Degree": "",
      "Position": ""
    }
  },
  {
   

In [77]:
df['start_year'].value_counts()

start_year
1993    1041
1991    1004
1992    1003
1989     938
1983     842
1980     809
1974     776
1988     734
1977     714
1986     706
1984     694
1973     684
1969     532
1972     522
1968     519
1971     466
1970     442
1978     415
1965     335
1964     306
1961     292
1959     284
1960     276
1958     267
1957     267
1985     221
1979     170
1962      86
1963      81
1987      74
1994      38
Name: count, dtype: int64

In [67]:
import pandas as pd
import json

# First, let's explore what formats exist
def safe_json_parse(text):
    """Try to parse JSON, return None if it fails"""
    try:
        return json.loads(text)
    except (json.JSONDecodeError, TypeError):
        return None

# Apply the parsing
df['content_json'] = df['content'].apply(safe_json_parse)

# Check how many successfully parsed
print(f"Successfully parsed: {df['content_json'].notna().sum()} / {len(df)}")
print(f"Failed to parse: {df['content_json'].isna().sum()}")

# Examine the failures
failed_mask = df['content_json'].isna()
if failed_mask.any():
    print("\n=== Sample of failed parses ===")
    print(df[failed_mask]['content'].head())
    
    # Check content types of failures
    print("\n=== Content types in failures ===")
    print(df[failed_mask]['content'].apply(type).value_counts())
    
    # Check first 100 characters of failed content
    print("\n=== First 100 chars of failed content ===")
    for idx in df[failed_mask].head().index:
        print(f"\nRow {idx}:")
        print(repr(df.loc[idx, 'content'][:100]))

Successfully parsed: 15091 / 15538
Failed to parse: 447

=== Sample of failed parses ===
43     [{"Department":"BIOLOGICAL SCIENCES","Number":...
76     [["Department","Number","Title","Description",...
85     [\n  {\n    "Department": "M.E.",\n    "Number...
86     [\n  {\n    "Department": "French",\n    "Numb...
131    [\n  {\n    "Department": "Linguistics",\n    ...
Name: content, dtype: object

=== Content types in failures ===
content
<class 'str'>    447
Name: count, dtype: int64

=== First 100 chars of failed content ===

Row 43:
'[{"Department":"BIOLOGICAL SCIENCES","Number":"291","Title":"al ADVANCEDEN","Description":"The descr'

Row 76:
'[["Department","Number","Title","Description","Prerequisites","Credits","Teacher"],\n["Anthropology",'

Row 85:
'[\n  {\n    "Department": "M.E.",\n    "Number": "122",\n    "Title": "Mechanical Engineering Laboratory'

Row 86:
'[\n  {\n    "Department": "French",\n    "Number": "",\n    "Title": "La civilisation française du XIXe '

Row 13

In [68]:
# Let's see the actual JSON parsing errors and string lengths
def parse_with_error(text):
    """Try to parse and return the error if it fails"""
    try:
        return json.loads(text)
    except json.JSONDecodeError as e:
        return {'error': str(e), 'position': e.pos, 'length': len(text)}
    except Exception as e:
        return {'error': str(e), 'length': len(text)}

# Check the failures in detail
failed_mask = df['content_json'].isna()
error_analysis = df[failed_mask]['content'].apply(parse_with_error).head(10)

for idx, error_info in zip(df[failed_mask].head(10).index, error_analysis):
    print(f"\n=== Row {idx} ===")
    if isinstance(error_info, dict) and 'error' in error_info:
        print(f"Length: {error_info['length']}")
        print(f"Error: {error_info['error']}")
        if 'position' in error_info:
            pos = error_info['position']
            # Show context around the error
            content = df.loc[idx, 'content']
            start = max(0, pos - 50)
            end = min(len(content), pos + 50)
            print(f"Context around position {pos}:")
            print(repr(content[start:end]))


=== Row 43 ===
Length: 3617
Error: Expecting ',' delimiter: line 1 column 1319 (char 1318)
Context around position 1318:
'":"3:15-4:30 alternate years, not given 1989-90"}}},{"Department":"BIOLOGICAL SCIENCES","Number":"10'

=== Row 76 ===
Length: 1653
Error: Expecting ',' delimiter: line 2 column 298 (char 381)
Context around position 381:
' within a particular cultural order.","","4-5","{"Honorific":"DR","Name":"A","Degree":"","Position":'

=== Row 85 ===
Length: 14316
Error: Expecting property name enclosed in double quotes: line 738 column 6 (char 14316)
Context around position 14316:
's": "",\n    "Credits": "6",\n    "Teacher": {\n     '

=== Row 86 ===
Length: 4112
Error: Expecting ',' delimiter: line 19 column 16 (char 550)
Context around position 550:
'ment": "French",\n    "Number": "",\n    "Title": ""La condition humaine": André Malraux",\n    "Descri'

=== Row 131 ===
Length: 14841
Error: Expecting property name enclosed in double quotes: line 676 column 1 (char 14841)
Co

In [69]:
import json
import re

def robust_json_parser(text):
    """Handle multiple JSON parsing issues found in the data"""
    if pd.isna(text) or text == '':
        return None
    
    # Strategy 1: Direct parse (works for 97% of data)
    try:
        return json.loads(text)
    except:
        pass
    
    # Strategy 2: Handle concatenated JSON arrays (Row 126)
    # Pattern: }]\n[{ or }][{
    if re.search(r'\]\s*\[', text):
        try:
            # Split on ][, parse each part, and combine
            parts = re.split(r'\]\s*\[', text)
            if len(parts) > 1:
                # Add back the brackets
                arrays = []
                arrays.append(json.loads(parts[0] + ']'))
                for part in parts[1:-1]:
                    arrays.append(json.loads('[' + part + ']'))
                arrays.append(json.loads('[' + parts[-1]))
                # Flatten all arrays into one
                result = []
                for arr in arrays:
                    result.extend(arr)
                return result
        except:
            pass
    
    # Strategy 3: Fix double newlines breaking structure (Row 230)
    try:
        fixed = re.sub(r'},\s*\n\s*\n\s*{', '}, {', text)
        return json.loads(fixed)
    except:
        pass
    
    # Strategy 4: Fix truncated JSON (Row 103)
    try:
        fixed = text.rstrip()
        # Count unmatched braces/brackets
        open_braces = fixed.count('{') - fixed.count('}')
        open_brackets = fixed.count('[') - fixed.count(']')
        open_quotes = fixed.count('"') % 2
        
        # If we're in an incomplete string, close it
        if open_quotes == 1:
            fixed += '"'
        
        # Close any open structures
        fixed += '}' * open_braces
        fixed += ']' * open_brackets
        
        return json.loads(fixed)
    except:
        pass
    
    # Strategy 5: Fix malformed endings (Row 364)
    # Pattern: extra quote before closing brace
    try:
        fixed = re.sub(r'",\s*""\}', '"}', text)
        return json.loads(fixed)
    except:
        pass
    
    # Strategy 6: Clean up weird character sequences (Row 562)
    try:
        # Remove repeated single characters with spaces
        fixed = re.sub(r'(\s\d){10,}', '', text)
        return json.loads(fixed)
    except:
        pass
    
    # Return None if all strategies fail
    return None

# Apply the parser
df['content_parsed'] = df['content'].apply(robust_json_parser)

print(f"Successfully parsed: {df['content_parsed'].notna().sum()} / {len(df)}")
print(f"Still failed: {df['content_parsed'].isna().sum()}")

# Check remaining failures
still_failed = df['content_parsed'].isna()
if still_failed.any():
    print("\nRemaining failures:")
    print(df[still_failed]['content'].head(3).tolist())

Successfully parsed: 15169 / 15538
Still failed: 369

Remaining failures:
['[{"Department":"BIOLOGICAL SCIENCES","Number":"291","Title":"al ADVANCEDEN","Description":"The descriptions refer to prerequisite courses from the core sequence by the old and new numbers. Students who took the core lecture courses between 1980-81 and 1985-86 should follow the guidelines listed under courses 31, 32, 33 after the prerequisites.","Prerequisites":"Prerequisites: 33 or 40, or equivalent.","Credits":"-3 units","Teacher":{"Honorific":"","Name":"Baxter","Degree":"","Position":"MT"}},{"Department":"BIOLOGICAL SCIENCES","Number":"291","Title":"","Description":"","Prerequisites":"","Credits":"","Teacher":{}},{"Department":"BIOLOGICAL SCIENCES","Number":"102","Title":"Biology of Marine Communities","Description":"Treats a set of marine communities by considering (1) physical environment, (2) resident species, (3) their biology and interactions, (4) population and community structure and dynamics, and (5) 

In [91]:
df = df[df['identified_type'].isin(['course_listing'])]

df = df.reset_index(drop=True)

In [93]:
# 1. Check the 'type' column - it likely indicates what kind of content
print(df['type'].value_counts())

# 2. Look at examples of each type
for content_type in df['type'].unique():
    print(f"\n{'='*50}")
    print(f"Type: {content_type}")
    print(f"{'='*50}")
    sample = df[df['type'] == content_type]['content_parsed'].iloc[0]
    if isinstance(sample, list) and len(sample) > 0:
        print(f"First item keys: {sample[0].keys()}")
        print(f"Example:\n{sample[0]}")

# 3. If type column doesn't distinguish, check the structure of content_parsed
def identify_content_type(parsed_content):
    """Identify if content is courses, faculty, or mixed based on dict keys"""
    if not isinstance(parsed_content, list) or len(parsed_content) == 0:
        return 'empty'
    
    first_item = parsed_content[0]
    if not isinstance(first_item, dict):
        return 'unknown'
    
    keys = set(first_item.keys())
    
    # Check for course-specific keys
    if 'Number' in keys or 'Credits' in keys or 'Description' in keys:
        return 'course_listing'
    # Check for faculty-specific keys (you'll need to adjust based on actual structure)
    elif 'Faculty' in keys or 'Position' in keys:
        return 'faculty_listing'
    else:
        return 'other'

df['identified_type'] = df['content_parsed'].apply(identify_content_type)
print("\nIdentified content types:")
print(df['identified_type'].value_counts())

# 4. Show examples of each identified type
for identified_type in df['identified_type'].unique():
    print(f"\n{'='*50}")
    print(f"Identified as: {identified_type}")
    sample_row = df[df['identified_type'] == identified_type].iloc[0]
    print(f"Actual 'type' column: {sample_row['type']}")
    if isinstance(sample_row['content_parsed'], list) and len(sample_row['content_parsed']) > 0:
        print(f"Sample item:\n{sample_row['content_parsed'][0]}")

type
courses    14445
Name: count, dtype: int64

Type: courses
First item keys: dict_keys(['Department', 'Number', 'Title', 'Description', 'Prerequisites', 'Credits', 'Teacher'])
Example:
{'Department': 'VALUES, TECHNOLOGY', 'Number': '151', 'Title': 'SCIENCE AND SOCIETY', 'Description': 'trol, the H-bomb decision and tactical weapons, disputes over nuclear testing and fallout, the development of deterrence and its critics, the roles of scientists and the strategy community, and various peace movements.', 'Prerequisites': '', 'Credits': '4-5', 'Teacher': {'Honorific': '', 'Name': 'Bernstein', 'Degree': '', 'Position': 'Professor'}}

Identified content types:
identified_type
course_listing    14445
Name: count, dtype: int64

Identified as: course_listing
Actual 'type' column: courses
Sample item:
{'Department': 'VALUES, TECHNOLOGY', 'Number': '151', 'Title': 'SCIENCE AND SOCIETY', 'Description': 'trol, the H-bomb decision and tactical weapons, disputes over nuclear testing and fallout, 

In [106]:
# Explode the content_parsed column so each course gets its own row
df_courses = df.explode('content_parsed').reset_index(drop=True)

# Now each row has the same metadata but content_parsed contains a single course dict
# If you want to expand the course dictionary into separate columns:
df_courses = df_courses.dropna(subset=['content_parsed'])  # Remove rows with no parsed content

# Extract course fields into separate columns
df_courses['Department'] = df_courses['content_parsed'].apply(lambda x: x.get('Department', '') if isinstance(x, dict) else '')
df_courses['Number'] = df_courses['content_parsed'].apply(lambda x: x.get('Number', '') if isinstance(x, dict) else '')
df_courses['Title'] = df_courses['content_parsed'].apply(lambda x: x.get('Title', '') if isinstance(x, dict) else '')
df_courses['Description'] = df_courses['content_parsed'].apply(lambda x: x.get('Description', '') if isinstance(x, dict) else '')
df_courses['Prerequisites'] = df_courses['content_parsed'].apply(lambda x: x.get('Prerequisites', '') if isinstance(x, dict) else '')
df_courses['Credits'] = df_courses['content_parsed'].apply(lambda x: x.get('Credits', '') if isinstance(x, dict) else '')



# drop the content_parsed column
df_courses = df_courses.drop('content_parsed', axis=1)
df_courses = df_courses.drop('type', axis=1)
df_courses = df_courses.drop('content', axis=1)

df_courses.to_csv('../data/uvm_courses.csv', index=False)



In [110]:
df_courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138839 entries, 0 to 138838
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               138839 non-null  int64 
 1   ipeds_code       138839 non-null  int64 
 2   level            138839 non-null  object
 3   start_year       138839 non-null  int64 
 4   end_year         138839 non-null  int64 
 5   university_name  138839 non-null  object
 6   content_json     136915 non-null  object
 7   identified_type  138839 non-null  object
 8   Department       138839 non-null  object
 9   Number           138837 non-null  object
 10  Title            138837 non-null  object
 11  Description      138740 non-null  object
 12  Prerequisites    138765 non-null  object
 13  Credits          138774 non-null  object
dtypes: int64(4), object(10)
memory usage: 14.8+ MB
